## This example demonstrates XCSF hyperparameter tuning

In [1]:
import json

import numpy as np
import optuna
from optuna.samplers import TPESampler
from sklearn.compose import TransformedTargetRegressor
from sklearn.datasets import fetch_openml
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from optuna.integration import OptunaSearchCV
from optuna.distributions import FloatDistribution

import xcsf

RANDOM_STATE: int = 1

np.random.seed(RANDOM_STATE)

### Load training and test data
Loads the kin8nm dataset from: https://www.openml.org/d/189

"This is data set is concerned with the forward kinematics of an 8 link robot arm. Among the existing variants of this data set we have used the variant 8nm, which is known to be highly non-linear and medium noisy." data

In [2]:
data = fetch_openml(data_id=189, as_frame=True, parser="auto")

data.frame.head()

,theta1,theta2,theta3,theta4,theta5,theta6,theta7,theta8,y
0,-0.015119,0.360741,0.469398,1.309675,0.988024,-0.025493,0.664071,0.062763,0.536524
1,0.360478,-0.301395,0.629183,-1.440146,-0.741637,-1.196749,-1.038444,-0.717461,0.308014
2,1.563238,-1.294753,0.078987,1.432937,1.149136,-1.292140,1.562988,-0.937731,0.518900
3,0.199485,0.901157,-1.356304,-0.080525,-0.976628,0.829894,-0.855649,0.930630,0.494151
4,0.659737,0.120552,-0.008756,0.648839,0.626832,-0.646539,1.318074,-0.899172,0.470218


In [3]:
data.frame.describe()

,theta1,theta2,theta3,theta4,theta5,theta6,theta7,theta8,y
count,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000
mean,0.009351,-0.007233,0.022792,-0.012945,0.007886,-0.009698,0.008402,0.001227,0.714283
std,0.904867,0.901496,0.900765,0.899186,0.906874,0.913243,0.909724,0.906098,0.263607
min,-1.570611,-1.570681,-1.570669,-1.570295,-1.570664,-1.570254,-1.570643,-1.568904,0.040165
25%,-0.770328,-0.778319,-0.749718,-0.780602,-0.770492,-0.802084,-0.776870,-0.777432,0.523584
50%,-0.002010,-0.015152,0.041346,-0.013617,0.009702,-0.003575,0.001338,0.002007,0.708822
75%,0.791179,0.779145,0.792391,0.760165,0.791203,0.781849,0.798014,0.780927,0.901354
max,1.570753,1.570657,1.570736,1.570169,1.570599,1.570597,1.570522,1.570237,1.458521


In [4]:
# numpy
X = np.asarray(data.data, dtype=np.float64)
y = np.asarray(data.target, dtype=np.float64)

# normalise inputs (zero mean and unit variance)
scaler = StandardScaler()
scaler.fit_transform(X)

# reshape outputs into 2D arrays
if len(np.shape(y)) == 1:
    y = y.reshape(-1, 1)

# scale outputs [0,1]
minmax = MinMaxScaler(feature_range=(0, 1))
y = minmax.fit_transform(y)

# split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=RANDOM_STATE
)

# 10% of training for validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=RANDOM_STATE
)

print(f"X_train shape = {np.shape(X_train)}")
print(f"y_train shape = {np.shape(y_train)}")
print(f"X_val shape = {np.shape(X_val)}")
print(f"y_val shape = {np.shape(y_val)}")
print(f"X_test shape = {np.shape(X_test)}")
print(f"y_test shape = {np.shape(y_test)}")

X_DIM: int = np.shape(X_train)[1]
Y_DIM: int = np.shape(y_train)[1]

X_train shape = (6634, 8)
y_train shape = (6634, 1)
X_val shape = (738, 8)
y_val shape = (738, 1)
X_test shape = (820, 8)
y_test shape = (820, 1)


### Initialise XCSF

In [5]:
# Due to the way XCSF initialises conditions, actions, and predictions,
# it is easier to set the desired parameters in a wrapper function...


def get_prediction(n_init: int) -> dict:
    """Returns parameters for initialising neural network predictions."""
    return {
        "type": "neural",
        "args": {
            "layer_0": {
                "type": "connected",
                "activation": "relu",
                "n_init": n_init,
                "evolve_weights": True,
                "evolve_functions": False,
                "evolve_connect": True,
                "evolve_neurons": False,
                "sgd_weights": True,
                "eta": 0.1,
                "evolve_eta": True,
                "eta_min": 1e-06,
                "momentum": 0.9,
                "decay": 0,
            },
            "layer_1": {
                "type": "connected",
                "activation": "softplus",
                "n_init": Y_DIM,
                "evolve_weights": True,
                "evolve_functions": False,
                "evolve_connect": True,
                "evolve_neurons": False,
                "sgd_weights": True,
                "eta": 0.1,
                "evolve_eta": True,
                "eta_min": 1e-06,
                "momentum": 0.9,
                "decay": 0,
            },
        },
    }


def get_model() -> xcsf.XCS:
    """Returns a new XCS with baseline parameters."""
    return xcsf.XCS(
        x_dim=X_DIM,
        y_dim=Y_DIM,
        n_actions=1,
        omp_num_threads=12,
        random_state=RANDOM_STATE,
        pop_init=True,
        max_trials=200000,
        perf_trials=5000,
        pop_size=500,
        loss_func="mse",
        set_subsumption=False,
        theta_sub=100,
        e0=0.005,
        alpha=1,
        nu=20,
        beta=0.1,
        delta=0.1,
        theta_del=50,
        init_fitness=0.01,
        init_error=0,
        m_probation=10000,
        stateful=True,
        compaction=False,
        ea={
            "select_type": "roulette",
            "theta_ea": 50,
            "lambda": 2,
            "p_crossover": 0.8,
            "err_reduc": 1,
            "fit_reduc": 0.1,
            "subsumption": False,
            "pred_reset": False,
        },
        condition={
            "type": "tree_gp",
            "args": {
                "min_constant": 0,
                "max_constant": 1,
                "n_constants": 100,
                "init_depth": 5,
                "max_len": 10000,
            },
        },
        prediction=get_prediction(1),
    )

In [6]:
def get_callback():
    """Returns a new callback for early stoppping."""
    return xcsf.EarlyStoppingCallback(
        # note: PERF_TRIALS is considered an "epoch" for callbacks
        monitor="val",  # which metric to monitor: {"train", "val"}
        patience=20000,  # trials with no improvement after which training will be stopped
        restore_best=True,  # whether to make checkpoints and restore best population
        min_delta=0,  # minimum change to qualify as an improvement
        start_from=0,  # trials to wait before starting to monitor improvement
        verbose=True,  # whether to display when checkpoints are made
    )

### Optuna

In [7]:
def objective(trial):
    """Measure a new hyperparameter combination."""
    # get new parameters to try
    e0: float = trial.suggest_float("e0", 0.001, 0.01, step=0.001)
    n_init: int = trial.suggest_int("n_init", 1, 41, step=4)
    # get a new XCSF and set params
    prediction: dict = get_prediction(n_init=n_init)
    model = get_model()
    model.set_params(e0=e0, prediction=prediction)
    # get a new early stopping callback
    callback = get_callback()
    # fit XCSF
    model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        callbacks=[callback],
        verbose=True,
    )
    # optimise validation MSE
    return model.score(X_val, y_val)


study = optuna.create_study(direction="minimize", sampler=TPESampler(seed=RANDOM_STATE))
study.optimize(objective, n_trials=10)

[I 2023-07-18 17:00:51,136] A new study created in memory with name: no-name-6d2cd670-8bc5-4242-bb99-4d1ce347d56d


00:00:00.656 trials=0 train=0.01245 val=0.01051 pset=500.0 mset=218.4 mfrac=0.20
checkpoint: 0.01051 error at 0 trials
00:00:00.834 trials=5000 train=0.00801 val=0.00841 pset=500.0 mset=226.8 mfrac=0.49
checkpoint: 0.00841 error at 5000 trials
00:00:01.079 trials=10000 train=0.00705 val=0.00754 pset=500.0 mset=384.5 mfrac=1.00
checkpoint: 0.00754 error at 10000 trials
00:00:01.289 trials=15000 train=0.00587 val=0.00696 pset=500.0 mset=378.5 mfrac=1.00
checkpoint: 0.00696 error at 15000 trials
00:00:01.652 trials=20000 train=0.00557 val=0.00628 pset=500.0 mset=364.6 mfrac=1.00
checkpoint: 0.00628 error at 20000 trials
00:00:01.405 trials=25000 train=0.00495 val=0.00592 pset=500.0 mset=383.8 mfrac=1.00
checkpoint: 0.00592 error at 25000 trials
00:00:01.093 trials=30000 train=0.00486 val=0.00579 pset=500.0 mset=376.3 mfrac=1.00
checkpoint: 0.00579 error at 30000 trials
00:00:01.068 trials=35000 train=0.00456 val=0.00543 pset=500.0 mset=369.7 mfrac=1.00
checkpoint: 0.00543 error at 35000 t

[I 2023-07-18 17:02:05,009] Trial 0 finished with value: 0.004053235508717323 and parameters: {'e0': 0.005, 'n_init': 29}. Best is trial 0 with value: 0.004053235508717323.


00:00:02.115 trials=195000 train=0.00323 val=0.00405 pset=500.0 mset=453.2 mfrac=1.00
checkpoint: 0.00405 error at 195000 trials
restoring system from trial 195000 with error=0.00405
00:01:13.592
00:00:03.113 trials=0 train=0.01428 val=0.01227 pset=500.0 mset=309.0 mfrac=0.70
checkpoint: 0.01227 error at 0 trials
00:00:01.141 trials=5000 train=0.00976 val=0.00998 pset=500.0 mset=367.9 mfrac=0.33
checkpoint: 0.00998 error at 5000 trials
00:00:01.400 trials=10000 train=0.00859 val=0.00973 pset=500.0 mset=443.1 mfrac=0.50
checkpoint: 0.00973 error at 10000 trials
00:00:02.790 trials=15000 train=0.00706 val=0.00874 pset=500.0 mset=434.2 mfrac=0.26
checkpoint: 0.00874 error at 15000 trials
00:00:01.291 trials=20000 train=0.00634 val=0.00898 pset=500.0 mset=420.7 mfrac=0.22
00:00:01.312 trials=25000 train=0.00581 val=0.00704 pset=500.0 mset=415.8 mfrac=0.43
checkpoint: 0.00704 error at 25000 trials
00:00:01.319 trials=30000 train=0.00584 val=0.00659 pset=500.0 mset=411.8 mfrac=0.02
checkpoin

[I 2023-07-18 17:02:34,144] Trial 1 finished with value: 0.005653397062139404 and parameters: {'e0': 0.001, 'n_init': 13}. Best is trial 0 with value: 0.004053235508717323.


...

00:00:00.855 trials=130000 train=0.00332 val=0.00447 pset=500.0 mset=34.8 mfrac=0.03
restoring system from trial 105000 with error=0.00415
00:00:36.907
00:00:01.521 trials=0 train=0.01270 val=0.01149 pset=500.0 mset=196.5 mfrac=0.68
checkpoint: 0.01149 error at 0 trials
00:00:02.517 trials=5000 train=0.00850 val=0.00871 pset=500.0 mset=217.8 mfrac=0.02
checkpoint: 0.00871 error at 5000 trials
00:00:01.211 trials=10000 train=0.00698 val=0.00738 pset=500.0 mset=377.7 mfrac=0.17
checkpoint: 0.00738 error at 10000 trials
00:00:01.615 trials=15000 train=0.00615 val=0.00772 pset=500.0 mset=395.3 mfrac=1.00
00:00:02.032 trials=20000 train=0.00536 val=0.00598 pset=500.0 mset=406.9 mfrac=1.00
checkpoint: 0.00598 error at 20000 trials
00:00:01.875 trials=25000 train=0.00491 val=0.00681 pset=500.0 mset=402.4 mfrac=1.00
00:00:01.738 trials=30000 train=0.00471 val=0.00564 pset=500.0 mset=403.3 mfrac=0.79
checkpoint: 0.00564 error at 30000 trials
00:00:01.697 trials=35000 train=0.00474 val=0.00530 p

[I 2023-07-18 17:08:47,812] Trial 8 finished with value: 0.004309360713996344 and parameters: {'e0': 0.005, 'n_init': 25}. Best is trial 6 with value: 0.003864337147497149.


00:00:01.886 trials=140000 train=0.00347 val=0.00432 pset=500.0 mset=458.9 mfrac=1.00
restoring system from trial 115000 with error=0.00431
00:00:57.371
00:00:00.866 trials=0 train=0.01397 val=0.01197 pset=500.0 mset=270.1 mfrac=1.00
checkpoint: 0.01197 error at 0 trials
00:00:00.883 trials=5000 train=0.00985 val=0.00955 pset=500.0 mset=268.5 mfrac=1.00
checkpoint: 0.00955 error at 5000 trials
00:00:01.055 trials=10000 train=0.00869 val=0.00961 pset=500.0 mset=366.3 mfrac=0.00
00:00:01.181 trials=15000 train=0.00748 val=0.00833 pset=500.0 mset=349.6 mfrac=0.01
checkpoint: 0.00833 error at 15000 trials
00:00:01.120 trials=20000 train=0.00662 val=0.00757 pset=500.0 mset=311.6 mfrac=0.03
checkpoint: 0.00757 error at 20000 trials
00:00:01.003 trials=25000 train=0.00621 val=0.00678 pset=500.0 mset=275.5 mfrac=0.49
checkpoint: 0.00678 error at 25000 trials
00:00:00.945 trials=30000 train=0.00583 val=0.00646 pset=500.0 mset=251.4 mfrac=0.11
checkpoint: 0.00646 error at 30000 trials
00:00:00.8

[I 2023-07-18 17:09:00,906] Trial 9 finished with value: 0.0061014338323500126 and parameters: {'e0': 0.002, 'n_init': 9}. Best is trial 6 with value: 0.003864337147497149.


00:00:00.515 trials=75000 train=0.00485 val=0.00633 pset=500.0 mset=26.2 mfrac=0.13
restoring system from trial 50000 with error=0.00610
00:00:12.961


In [8]:
best_trial = study.best_trial
print(f"Best trial - Evaluation metric: {best_trial.value:.4f}")
print(f"Best trial - Hyperparameters: {best_trial.params}")

Best trial - Evaluation metric: 0.0039
Best trial - Hyperparameters: {'e0': 0.003, 'n_init': 37}


### Sklearn pipeline

Note: sklearn pipelines do not allow the use of callback functions.

In [9]:
model = get_model()

model = make_pipeline(
    MinMaxScaler(feature_range=(-1.0, 1.0)),
    TransformedTargetRegressor(regressor=model, transformer=StandardScaler()),
)

model.fit(X_train, y_train)

00:00:00.630 trials=0 train=0.83586 pset=500.0 mset=310.1 mfrac=0.78
00:00:00.804 trials=5000 train=0.81021 pset=500.0 mset=348.1 mfrac=0.84
00:00:00.958 trials=10000 train=0.82649 pset=500.0 mset=443.1 mfrac=0.81
00:00:01.179 trials=15000 train=0.79003 pset=500.0 mset=439.6 mfrac=0.55
00:00:01.109 trials=20000 train=0.81252 pset=500.0 mset=446.9 mfrac=0.90
00:00:01.363 trials=25000 train=0.80616 pset=500.0 mset=449.8 mfrac=0.91
00:00:01.150 trials=30000 train=0.80414 pset=500.0 mset=447.1 mfrac=0.60
00:00:02.429 trials=35000 train=0.80525 pset=500.0 mset=453.3 mfrac=0.66
00:00:01.286 trials=40000 train=0.81692 pset=500.0 mset=455.5 mfrac=0.17
00:00:01.321 trials=45000 train=0.79554 pset=500.0 mset=453.6 mfrac=0.85
00:00:01.254 trials=50000 train=0.82586 pset=500.0 mset=460.0 mfrac=1.00
00:00:01.196 trials=55000 train=0.82046 pset=500.0 mset=455.4 mfrac=0.71
00:00:01.199 trials=60000 train=0.79368 pset=500.0 mset=457.1 mfrac=0.98
00:00:01.206 trials=65000 train=0.80103 pset=500.0 mset=

Pipeline(steps=[('minmaxscaler', MinMaxScaler(feature_range=(-1.0, 1.0))),
                ('transformedtargetregressor',
                 TransformedTargetRegressor(regressor=<xcsf.xcsf.XCS object at 0x7f17bc31a370>,
                                            transformer=StandardScaler()))])

In [10]:
y_pred = model.predict(X_test)

train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"train = {train_score}")
print(f"test = {test_score}")

train = 0.1977731736200089
test = 0.16539292441508746


### Sklearn grid search

General parameters can be searched in the usual way:

In [11]:
parameters = {"beta": [0.1, 0.5]}

model = get_model()
model.set_params(max_trials=20000)

grid_search = GridSearchCV(model, parameters, scoring="neg_mean_squared_error")

grid_search.fit(X_train, y_train)

results = grid_search.cv_results_

for mean_score, std_score, params in zip(
    results["mean_test_score"], results["std_test_score"], results["params"]
):
    print("Mean Score:", -mean_score)
    print("Standard Deviation:", std_score)
    print("Parameters:", params)
    print("------------------------")

print("Best parameters: ", grid_search.best_params_)
print("Best score: ", -grid_search.best_score_)

00:00:00.734 trials=0 train=0.02151 pset=500.0 mset=226.1 mfrac=0.75
00:00:00.783 trials=5000 train=0.02072 pset=500.0 mset=202.9 mfrac=0.23
00:00:00.827 trials=10000 train=0.01977 pset=500.0 mset=304.6 mfrac=0.50
00:00:01.145 trials=15000 train=0.02031 pset=500.0 mset=297.7 mfrac=0.00
00:00:03.489
00:00:00.615 trials=0 train=0.02130 pset=500.0 mset=219.7 mfrac=0.47
00:00:00.650 trials=5000 train=0.01975 pset=500.0 mset=192.0 mfrac=0.06
00:00:00.867 trials=10000 train=0.01933 pset=500.0 mset=310.9 mfrac=0.00
00:00:00.798 trials=15000 train=0.01882 pset=500.0 mset=286.3 mfrac=0.11
00:00:02.930
00:00:00.684 trials=0 train=0.02129 pset=500.0 mset=230.4 mfrac=0.12
00:00:01.319 trials=5000 train=0.02040 pset=500.0 mset=176.3 mfrac=0.69
00:00:00.818 trials=10000 train=0.01950 pset=500.0 mset=273.6 mfrac=0.06
00:00:00.939 trials=15000 train=0.01961 pset=500.0 mset=216.2 mfrac=0.01
00:00:03.760
00:00:00.951 trials=0 train=0.02125 pset=500.0 mset=213.2 mfrac=0.74
00:00:00.752 trials=5000 train=

EA parameters require specifying a dict, but individual values can still be set because the other values are not changed.

In [12]:
parameters = {"ea": [{"lambda": 2}, {"lambda": 10}, {"lambda": 50}]}

However, for actions, conditions, and predictions, the WHOLE dict must be specified
for each value to try in the search. This is because of the way XCSF uses kwargs to
initialise values and they are reset each time. XCSF has so many different parameters
that it is unfortunately necessary to do it this way.